# Download and split raw data into training and test data    
Christoph Windheuser, ThoughtWorks, June 19, 2020    
     
This notebook needs to be run in SageMaker Studio. It reads the data as csv-file from a public S3 bucket. Then it splits the data into a training and a test set and saves all in a personal S3 bucket.

## Import the necessary libraries    
pandas is a python data science library to handle dataframes    
boto3 is the Amazon Web Services SDK for Python. It enables Python developers to create, configure, and manage AWS services, such as EC2 and S3.    
S3Uploader and S3Downloader are routines to upload or download data into S3.

In [31]:
import os
import pandas as pd
import json
from   sklearn.preprocessing import LabelEncoder
import boto3
from   sagemaker.s3 import S3Uploader, S3Downloader


---
## Definitions

In [2]:
train_filename   = 'store47-2016-train.csv'
test_filename    = 'store47-2016-test.csv'
final_train_file = 'final_train.csv'
final_test_file  = 'final_validate.csv'
local_data_dir   = 'CD4ML-AWS-Serverless/data'
local_tmp_dir    = 'CD4ML-AWS-Serverless/data/tmp'
s3_prefix        = 'demandforecast'
raw_filename     = 'store47-2016.csv'
s3_raw_data_path = 'https://christoph-windheuser-public.s3.amazonaws.com'


## Do some preparations

In [3]:
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page


## Define write_df_to_s3

In [4]:
def write_df_to_s3(df, filename, s3_path):
    if not os.path.exists(local_tmp_dir):
        os.makedirs(local_tmp_dir)
    df.to_csv('{}/{}'.format(local_tmp_dir, filename), index=False)
    s3url = S3Uploader.upload('{}/{}'.format(local_tmp_dir, filename), s3_path)
    os.remove('{}/{}'.format(local_tmp_dir, filename))
    print(s3url)


## Define write_dic_to_json_to_s3

In [36]:
def write_dic_to_json_to_s3(dic, filename, s3_path):
    if not os.path.exists(local_tmp_dir):
        os.makedirs(local_tmp_dir)
    
    with open('{}/{}'.format(local_tmp_dir, filename), 'w') as fp:
        json.dump(dic, fp)
        
    s3url = S3Uploader.upload('{}/{}'.format(local_tmp_dir, filename), s3_path)
    os.remove('{}/{}'.format(local_tmp_dir, filename))
    print(s3url)


---
## Open S3 Session and define Bucket

In [5]:
sess       = boto3.Session()
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()["Account"]
bucket     = 'sagemaker-studio-{}-{}'.format(sess.region_name, account_id)

try:
    if sess.region_name == "us-east-1":
        sess.client('s3').create_bucket(Bucket=bucket)
    else:
        sess.client('s3').create_bucket(Bucket=bucket, 
                                        CreateBucketConfiguration={'LocationConstraint': sess.region_name})
except Exception as e:
    print("Looks like you already have a bucket of this name. That's good. Uploading the data files...")


## Read raw data file

In [6]:
data = pd.read_csv('{}/{}'.format(s3_raw_data_path, raw_filename))


In [7]:
data.head(10)

,id,date,item_nbr,unit_sales,family,class,perishable,transactions,year,month,day,dayofweek,days_til_end_of_data,dayoff
0,88219279,2016-08-16,103520,10.0,GROCERY I,1028,0,3570,2016,8,16,1,364,False
1,88219280,2016-08-16,103665,4.0,BREAD/BAKERY,2712,1,3570,2016,8,16,1,364,False
2,88219281,2016-08-16,105574,9.0,GROCERY I,1045,0,3570,2016,8,16,1,364,False
3,88219282,2016-08-16,105575,45.0,GROCERY I,1045,0,3570,2016,8,16,1,364,False
4,88219283,2016-08-16,105577,8.0,GROCERY I,1045,0,3570,2016,8,16,1,364,False
5,88219284,2016-08-16,105693,2.0,GROCERY I,1034,0,3570,2016,8,16,1,364,False
6,88219285,2016-08-16,105737,6.0,GROCERY I,1044,0,3570,2016,8,16,1,364,False
7,88219286,2016-08-16,105857,14.0,GROCERY I,1092,0,3570,2016,8,16,1,364,False
8,88219287,2016-08-16,106716,13.0,GROCERY I,1032,0,3570,2016,8,16,1,364,False
9,88219288,2016-08-16,108079,2.0,GROCERY I,1030,0,3570,2016,8,16,1,364,False


## Split into train and test data set

In [8]:
# Split the data at the date 2017-08-02 (last 14 days of data set)
data_train = data[data['date'] < '2017-08-02']
data_test  = data[data['date'] >= '2017-08-02']


## Save train and test data as csv-file to S3

First, save files locally on the SageMaker instance:

In [9]:
write_df_to_s3(data_train, train_filename, 's3://{}/{}/{}'.format(bucket, s3_prefix,'train'))
write_df_to_s3(data_test,  test_filename,  's3://{}/{}/{}'.format(bucket, s3_prefix,'test'))


s3://sagemaker-studio-us-east-1-261586618408/demandforecast/train/store47-2016-train.csv
s3://sagemaker-studio-us-east-1-261586618408/demandforecast/test/store47-2016-test.csv


---
## Encode non-numerical values and drop date column
The encoding schema of the product families is written to the json-file "family_encoder.json", as this encoding is later necessary in the ML model for inference. This file is written to S3 into the directory "train/final/".

In [39]:
def join_tables(train, validate):
    print("Joining tables for consistent encoding")
    return train.append(validate).drop('date', axis=1)


def encode_categorical_columns(df):
    obj_df = df.select_dtypes(include=['object', 'bool']).copy().fillna('-1')
    lb = LabelEncoder()
    classes_dic = {}
    
    for col in obj_df.columns:
        print (col)
        df[col] = lb.fit_transform(obj_df[col])

        if col == "family":    
            classes = list(lb.classes_)
            for index, c in enumerate(classes):
                classes_dic[c] = index
            write_dic_to_json_to_s3(classes_dic, 'family_encoder.json', 's3://{}/{}/{}'.format(bucket, s3_prefix,'train/final'))

            # print (classes_dic)
    
    return df


def encode(train, validate):
    print("Encoding categorical variables")
    train_ids = train.id
    validate_ids = validate.id

    joined  = join_tables(train, validate)

    encoded = encode_categorical_columns(joined.fillna(-1))

    print("Not predicting returns (changing negative unit sales to 0)")
    encoded.loc[encoded.unit_sales < 0, 'unit_sales'] = 0

    validate = encoded[encoded['id'].isin(validate_ids)]
    train = encoded[encoded['id'].isin(train_ids)]
    return train, validate


In [40]:
train, validate = encode(data_train, data_test)
train.head()


Encoding categorical variables
Joining tables for consistent encoding
family
s3://sagemaker-studio-us-east-1-261586618408/demandforecast/train/final/family_encoder.json
dayoff
Not predicting returns (changing negative unit sales to 0)


,id,item_nbr,unit_sales,family,class,perishable,transactions,year,month,day,dayofweek,days_til_end_of_data,dayoff
0,88219279,103520,10.0,11,1028,0,3570,2016,8,16,1,364,0
1,88219280,103665,4.0,4,2712,1,3570,2016,8,16,1,364,0
2,88219281,105574,9.0,11,1045,0,3570,2016,8,16,1,364,0
3,88219282,105575,45.0,11,1045,0,3570,2016,8,16,1,364,0
4,88219283,105577,8.0,11,1045,0,3570,2016,8,16,1,364,0


In [35]:
%pwd

'/root'

In [12]:
validate.head()

,id,item_nbr,unit_sales,family,class,perishable,transactions,year,month,day,dayofweek,days_til_end_of_data,dayoff
958783,124124002,96995,2.0,11,1093,0,3936,2017,8,2,2,13,0
958784,124124003,99197,1.0,11,1067,0,3936,2017,8,2,2,13,0
958785,124124004,103520,5.0,11,1028,0,3936,2017,8,2,2,13,0
958786,124124005,103665,1.0,4,2712,1,3936,2017,8,2,2,13,0
958787,124124006,105574,17.0,11,1045,0,3936,2017,8,2,2,13,0


---
## Save train and test data to S3

In [13]:
write_df_to_s3(train,    final_train_file, 's3://{}/{}/{}'.format(bucket, s3_prefix,'train/final'))
write_df_to_s3(validate, final_test_file,  's3://{}/{}/{}'.format(bucket, s3_prefix,'test/final'))


s3://sagemaker-studio-us-east-1-261586618408/demandforecast/train/final/final_train.csv
s3://sagemaker-studio-us-east-1-261586618408/demandforecast/test/final/final_validate.csv
